## RDP - Stone Soup Experiments
[Documentation](https://stonesoup.readthedocs.io/en/v1.4/auto_examples/readers/Custom_Pandas_Dataloader.html#sphx-glr-auto-examples-readers-custom-pandas-dataloader-py)

Stone Soup tutorials 1 & 2 with data reader code from Kaggle.
* Tutorial 1: Kalman Filter
* Tutorial 2: Extended Kalman Filter for non-linear models (using polar coordinates instead of cartesian)

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
from matplotlib import pyplot as plt
from math import ceil, pi


import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
sys.path.append('../../..')
import data_functions as dfunc
import visualizations as v
from ttt_ss_funcs import generate_timestamps, ADSBTruthReader, CSVReaderXY, CSVReaderPolar, plot_all
# sys.path.append("../stonesoup") # go to parent dir
# # from customFunctions import *

from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.base import Property
from stonesoup.types.detection import Detection
from stonesoup.plotter import AnimatedPlotterly

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.angle import Bearing
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath
from stonesoup.plotter import AnimatedPlotterly, Plotter, Plotterly

# Tracker Imports
from stonesoup.types.state import GaussianState
sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}

METERS_in_NM = 1852

## Get Data from BigQuery
* rdp_straight: short, straight flight path
* rdp_extended: longer flight path
* adsb_straight: truth for rdp_straight
* adsb_extended: truth for rdp_extended


In [ ]:
target_address = 10537421
# adsb_sql = f"""SELECT `timestamp`,
#         time_of_day, 
#         latitude, 
#         longitude, 
#         target_address,
#         flight_level, 
#         rho, 
#         theta
# FROM radar_data.adsb
# WHERE test_date = '2024-07-17'
# AND target_address={target_address}
# and latitude is not NULL
# AND rho<20
# ORDER BY `timestamp`"""

# adsb_straight = dfunc.query_to_df(adsb_sql)

# rdp_sql = f"""SELECT 
#         `timestamp`,
#         time_of_day,
#         cal, 
#         rho,
#         theta, 
#         x, 
#         y, 
#         field_note 
# FROM radar_data.rdp
# WHERE `timestamp` >= '{adsb_straight.timestamp.min().strftime("%Y-%m-%d %H:%M:%S")}'
# AND `timestamp` <= '{adsb_straight.timestamp.max().strftime("%Y-%m-%d %H:%M:%S")}'
# AND sortie_id=61
# AND rho >= {adsb_straight.rho.min()-0.5}
# AND rho <= {adsb_straight.rho.max()+0.5}
# AND theta >= {adsb_straight.theta.min()- 5}
# AND theta <= {adsb_straight.theta.max()+5}"""

# rdp_straight = dfunc.query_to_df(rdp_sql)
# rdp_straight.head()

## Save/Read  to/from CSV

In [ ]:
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
adsb_file = f'{data_dir}/adsb_straight.csv'
# adsb_straight.to_csv(adsb_file, index=False)
adsb_data = pd.read_csv(adsb_file)

adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data = adsb_data.loc[~adsb_data['timestamp'].isna()]
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data['timestamp'] = adsb_data['timestamp'].dt.tz_localize(None)
adsb_data['timestamp'] = adsb_data['timestamp'].astype('datetime64[us]')

rdp_file = f'{data_dir}/rdp_straight.csv'
# rdp_straight['timestamp'] = pd.to_datetime(rdp_straight['timestamp'], errors='coerce')
# rdp_straight['theta_rad'] = np.deg2rad(rdp_straight.theta)
# rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] = rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] - 2*pi 

# rdp_straight = rdp_straight.loc[~rdp_straight['timestamp'].isna()]
# rdp_straight.to_csv(rdp_file, index=False)
rdp_data = pd.read_csv(rdp_file)
rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)
rdp_data['timestamp'] = rdp_data['timestamp'].astype('datetime64[us]')

# Matched Plots
matched_csv = f'{data_dir}/rdp_matched.csv'
rdp_matched = pd.read_csv(matched_csv)
rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)
rdp_matched['timestamp'] = rdp_matched['timestamp'].astype('datetime64[us]')

start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()

print(f'ADSB: {len(adsb_data)}')
print(f'RDP: {len(rdp_data)}')

## Matched Data Set
To make things even simpler, I'll grab the set of matched data for this test plane. Then most of the plots should be "true" detections. Let's see how the tracker does with that.

In [ ]:
file_dir = 'C:/Users/ttrinter/OneDrive - cspeed.com (1)/Documents/Data/Travis/2024-07-17'
matched_file = '20240717_Travis_matched_rdp_61.xlsx'
matched_data = pd.read_excel(f'{file_dir}/{matched_file}')
matched_data = matched_data.loc[(matched_data.target_address==target_address) &
                                (matched_data.close_enough==True)]
matched_data.head()

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
# rdp_matched = matched_data[['timestamp_rdp',
#                             'cal_rdp',
#                             'rho_rdp',
#                             'theta_rdp']]

# rdp_matched['theta_rad'] = np.deg2rad(rdp_matched.theta_rdp)
# # rdp_matched.loc[rdp_matched.theta_rad>2*pi, 'theta_rad'] = rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] - 2*pi 
# rdp_matched.rename(columns={'rho_rdp': 'rho',
#                             'theta_rdp': 'theta', 
#                             'timestamp_rdp': 'timestamp', 
#                             'cal_rdp': 'cal'}, 
#                             inplace=True)

# rdp_matched['x'], rdp_matched['y'] = zip(*rdp_matched.apply(lambda x: dfunc.polar_to_cartesian(x.rho, x.theta), axis=1))

# # matched_csv = f'{data_dir}/rdp_matched.csv'
# # rdp_matched.to_csv(matched_csv, index=False)

# rdp_matched.head()

In [ ]:
rdp_matched.plot.scatter(x='x', y='y')
plt.grid()
plt.title("Matched RDP Plots")

In [ ]:
# adsb_data.to_csv(adsb_file, index=False)
# rdp_data.to_csv(rdp_file, index=False)

In [ ]:
print(f'rho: {rdp_data.rho.min()} - {rdp_data.rho.max()}')
print(f'theta: {rdp_data.theta.min()} - {rdp_data.theta.max()}')

In [ ]:
np.rad2deg(rdp_data.theta_rad).hist()

In [ ]:
# # Travis Location
# lat0, lon0, alt0 = 38.25049, -121.92474, 40

# class CSVReaderXY(DetectionReader):
#     rdp_file: str = Property(doc="File with the radar data.")
#     ndim_state: int = Property(default=4)
#     pos_mapping: Tuple[int, int] = Property(default=(0, 2))
#     # vel_mapping: Tuple[int, int] = Property(default=(1, 3))

#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#         # Tutorial 6 Approach
#         measurement_model = LinearGaussian(
#             ndim_state=4,
#             mapping=(0, 2),
#             noise_covar=np.array([[5, 0],
#                                 [0, 5]])
#             )
        
#         self.model=measurement_model

#     @BufferedGenerator.generator_method
#     def detections_gen(self):
#         with open(self.rdp_file, newline='') as csv_file:
#             for row in csv.DictReader(csv_file):
#                 if not row['timestamp']:
#                     continue

#                 timestamp = dateutil.parser.parse(row['timestamp'], ignoretz=True)
#                 # lat = float(row['latitude'])
#                 # lon = float(row['longitude']) 
#                 x = float(row['x'])*METERS_in_NM
#                 y = float(row['y'])*METERS_in_NM

#                 yield timestamp, {Detection(
#                     [x, y], timestamp=timestamp, 
#                     measurement_model=self.model)}


# class CSVReaderPolar(DetectionReader):
#     rdp_file: str = Property(doc="File with the radar data.")
#     ndim_state: int = Property(default=4)
#     pos_mapping: Tuple[int, int] = Property(default=(0, 2))

#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#         # Kaggle Approach
#         measurement_model = CartesianToBearingRange(ndim_state=self.ndim_state, 
#                                                  mapping=self.pos_mapping, 
#                                                  noise_covar=np.diag([np.radians(0.2), 1]))
        
#         self.model=measurement_model


#     @BufferedGenerator.generator_method
#     def detections_gen(self):
#         with open(self.rdp_file, newline='') as csv_file:
#             for row in csv.DictReader(csv_file):
#                 if not row['timestamp']:
#                     continue

#                 timestamp = dateutil.parser.parse(row['timestamp'], ignoretz=True)
#                 rho = float(row['rho'])*METERS_in_NM
#                 phi = 2*pi - float(row['theta_rad']) + pi/2

#                 metadata = {
#                     'cal': float(row['cal']),
#                     'sensor': 'RDU103'
#                     }

#                 yield timestamp, {Detection(
#                     [Bearing(phi), rho], timestamp=timestamp,
#                     # [Bearing(phi), rho, dx, dy], timestamp=timestamp,
#                     metadata=metadata, measurement_model=self.model)}



In [ ]:
# class ADSBTruthReader(GroundTruthReader):
#     adsb_file: str = Property(doc="File with the adsb data.")

#     @staticmethod
#     def single_ground_truth_reader(adsb_file, isset=True):
#         truth = GroundTruthPath()
#         with open(adsb_file, newline='') as csv_file:
#             for row in csv.DictReader(csv_file):
#                 lat = float(row['latitude'])
#                 lon = float(row['longitude'])
#                 alt = float(row['flight_level'])*100
#                 time = dateutil.parser.parse(row['timestamp'])
#                 if row['target_address'] != "":
#                     planename = row['target_address']
#                 x, y, z = geodetic2enu(lat, lon, alt, lat0, lon0, alt0)
#                 truth.append(GroundTruthState(
#                     [x, 0, y, 0, z, 0],
#                     timestamp=time,
#                     metadata={"id": planename}))
#             if isset:
#                 truth = {truth}
#         return truth

#     @classmethod
#     def multiple_ground_truth_reader(cls, filenames):
#         truths = set()
#         for filename in filenames:
#             truths.add(cls.single_ground_truth_reader(filename, isset=False))
#         return truths

#     @BufferedGenerator.generator_method
#     def groundtruth_paths_gen(self):
#         truths = self.multiple_ground_truth_reader([adsb_file])
#         yield None, truths


### Mimicing code from Kaggle...

In [ ]:
# def generate_timestamps(start_time, end_time):
#     total_seconds = (end_time - start_time).total_seconds()
#     return [start_time + timedelta(seconds=n) for n in range(ceil(total_seconds))]

In [ ]:
matched_data.sort_values('timestamp_rdp')[['timestamp_rdp', 'rho_rdp','theta_rdp']].head()

In [ ]:
# Detections
matched_xy = CSVReaderXY(matched_csv)
matched_polar = CSVReaderPolar(matched_csv)

# ADSB
adsb = ADSBTruthReader.multiple_ground_truth_reader([adsb_file])

dets = [next(iter(detection[1])) for detection in matched_xy.detections_gen()]

timestamps = generate_timestamps(start_time, end_time)
plot_all(dets, adsb, start_time, end_time, plot_type='animated')

## From Tutorial #1

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.predictor.kalman import KalmanPredictor, ExtendedKalmanPredictor 
from stonesoup.updater.kalman import KalmanUpdater, ExtendedKalmanUpdater

In [ ]:
# Measurement Model
measurement_model = LinearGaussian(
    ndim_state=4,   # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2), # Mapping measurement vector index to state index
    noise_covar=np.array([[5, 0 ],  
                          [0, 5]])
    )  #Covariance matrix for Gaussian PDF


# Transition Model
q_const = 50
q_x = q_const
q_y = q_const

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])
predictor = KalmanPredictor(transition_model)
updater = KalmanUpdater(measurement_model)

# Set Prior for the known track
# using the known starting values
# prior = GaussianState([[90], [-100], [-72], [100]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# using the location of the radar
prior = GaussianState([[0], [50], [0], [50]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)


start_time = min(timestamps)

from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track
track = Track()

for det in dets:
    prediction = predictor.predict(prior, timestamp=det.timestamp)
    # print(det.timestamp)
    hypothesis = SingleHypothesis(prediction, det)   # Group a prediction and measurement
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

In [ ]:
# plot_all(dets, adsb, start_time, end_time, q_const, track, plot_type='animated')
plot_all(dets, adsb, start_time, end_time, q_const, track, plot_type='static')

Changing the starting prior didn't change the results much at all - so not very sensitive to that! However, the first track point is always oddly somewhere between the radar and the actual first observation. Maybe this will clean up in later iterations of the process.

## Polar Coordinates
Trying again, but changing the process to read rho and theta and, maybe later also radial velocity.

In [ ]:
# Detections
meas_polar = CSVReaderPolar(matched_csv)
dets = [next(iter(detection[1])) for detection in meas_polar.detections_gen()]

plot_all(dets, adsb, start_time, end_time, plot_type='animated')

### Kalman Filtering Again

In [ ]:
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.updater.kalman import ExtendedKalmanUpdater
from stonesoup.models.transition.linear import ConstantVelocity

In [ ]:
measurement_model = CartesianToBearingRange(ndim_state=4, 
                                            mapping=(0,2), 
                                            noise_covar=np.diag([np.radians(0.2), 1])
                                            )

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

predictor = ExtendedKalmanPredictor(transition_model)

updater = ExtendedKalmanUpdater(measurement_model)

from stonesoup.types.state import GaussianState
prior = GaussianState([[90], [-100], [-72], [100]], np.diag([10, 0.5, 10, 0.5]), timestamp=start_time)
# prior = GaussianState([[0], [10], [0], [10]], np.diag([10, 0.5, 10, 0.5]), timestamp=start_time)

from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track

track = Track()
for measurement in dets:
    prediction = predictor.predict(prior, timestamp=measurement.timestamp)
    hypothesis = SingleHypothesis(prediction, measurement)  # Group a prediction and measurement
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

In [ ]:
# plot_all(dets, adsb, start_time, end_time, q_const, track, plot_type='animated')
plot_all(dets, adsb, start_time, end_time, q_const, track, plot_type='static')